<a href="https://colab.research.google.com/github/mckennpc/Human_MDH2/blob/main/MD_simulation_Step1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Introduction**

Welcome to the world of Molecular Dynamics! This lesson is here to teach you the basic protocol of running a molecular dynamics (MD) simulation, including a bit of Python coding. If you don't know how to code, don't worry! These lessons assume no prior knowledge of code or Python. The main purpose of this colab is to provide a short, educational molecular dynamics simulation.

## What is molecular dynamics?
Molecular dynamics or MD is a computational approach that simulates the movements of molecules. Typically we are simulating the motions of one molecule to illustrate and understand how populations of this molecule behave in laboratory or cellular environments. The theoretical underpinnings of MD are based in statistical mechanics and predicting the motions of spheres. As knowledge of chemical systems advanced and computers became more accessible, the spheres have become more molecular and information like hydrogen bonding and charges are included. Wikipedia has a good [timeline and description of MD](https://en.wikipedia.org/wiki/Molecular_dynamics).

## Why is MD useful?
Molecules are in motion! While we frequently say structure determines function, that is a bit of an oversimplification. A more complete description is that structure determines how the molecule moves which determines properties and function. The movements of molecules are connected to forming interactions with ligands and cofactors, catalytic activity, and other chemical properties. Thus a structure is a picture of a horse, while MD is a movie of the horse racing showing how fast the horse is and how each part of the horse is involved in winning the race.

[Short introductory video](https://www.loom.com/share/e3ed804926874bf4ab1f8025ddb5be12?sid=997813cd-9653-4fd5-bfc8-6e2e3dd361a6)

From this fundamental description, it is then possible to model how changes in the protein structure from ligand binding or sequence changes or post-translational modification affect the dynamics and function of the molecule.

## What will I get from this Colab?
This colab does the simulation and provides the files that form the "movie" of the motions of your molecule. In [further colabs](https://colab.research.google.com/drive/1gsqVx570FsOm7-aDZNslvHc4Kkq2Gp1v?usp=sharing), you will analyze the fundamental motions of your molecule.


---

##**Please make a copy of this colab for your personal use!!**

A few things to start:

1.   These lessons only work in Google Chrome
2.   If you want to save your progress, go to File> Save a Copy in Drive; then locate a spot in your Drive folder
3.   Clicking the "play" button to the top left of a code block runs the code. Sometimes you can see the code and interact with it. However, if the code is hidden  it is to run adminstrative tasks in the background and you do not need to worry unless you are interested.
4. Sometimes the code may be hidden from view. To view it, click the '>' on the left of the title, until it changes to 'v'. This will reveal the code in that line.

Here are the files you will need for this code to run:
*  A .pdb file containing a set of atomic coordinates of your molecule **.pdb** stands for a Protein Data Bank file. It is a file containing the three-dimensional structural data of your protein directly from the Protein Data Bank. If you run into errors with your PDB file, ask your instructor.

If you would like a tutorial on Python basics, please reference [this colab](https://colab.research.google.com/drive/1UVupXh23ArJp2F9vqTmkVh0JnjEaemi1?usp=sharing ). It is not necessary to do this, however if you are interested in learning more about coding it is a great resource to use!

If anything goes wrong as you are running this code and you would like to restart, click "Runtime" at the top of the screen, then press "Restart session". This will competely restart your run, so make sure that is what you want!

This code should take approximately 1 hour to run.

---

**Acknowledgments**
- Colab adapted and developed by **Angela Kayll, Angelina Sardelli, and Christopher Berndsen** at James Madison University (ver 1 2024) with input and feedback from students in CHEM260 and members of the [MCC community](https://www.mdhcures.org/).

- Adapted from Making-it-rain by **Pablo R. Arantes** ([@pablitoarantes](https://twitter.com/pablitoarantes)), **Marcelo D. Polêto** ([@mdpoleto](https://twitter.com/mdpoleto)), **Conrado Pedebos** ([@ConradoPedebos](https://twitter.com/ConradoPedebos)) and **Rodrigo Ligabue-Braun** ([@ligabue_braun](https://twitter.com/ligabue_braun)).

- [David Koes](https://github.com/dkoes) for his  [py3Dmol](https://3dmol.csb.pitt.edu/) plugin.


---
---
# **Getting started**

The first thing we need to do is install all necessary libraries and packages for our simulation, which in simple terms is code that is needed to run other code. To install the packages you need, click the PLAY button on the left. These packages will load some settings that are needed to run the rest of the code.

It is also important to indicate what the code means. Any line that begins with a '#' contains a comment on the code. If the code is shown, it is important to read the '#' to know what the code is and potentially how to run it.


In [ ]:
#@title **1a. Install Conda Colab**
#@markdown Press PLAY. It will restart the session, don't worry.
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
#@title **1b. Install dependencies**
#@markdown Only press PLAY **after** the Conda Colab has finished downloading. It will take a few minutes, please wait :-)
# install dependencies
import subprocess
import sys
subprocess.run("rm -rf /usr/local/conda-meta/pinned", shell=True)
subprocess.run("mamba install -c conda-forge ambertools -y", shell=True)
import pytraj as pt
subprocess.run("pip -q install py3Dmol", shell=True)
subprocess.run("pip install git+https://github.com/pablo-arantes/biopandas", shell=True)
subprocess.run("mamba install openmm=7.7.0", shell=True)
subprocess.run("pip install --upgrade MDAnalysis==2.4.2", shell=True)

#load dependencies
import sys
from biopandas.pdb import PandasPdb
import openmm as mm
from openmm import *
from openmm.app import *
from openmm.unit import *
import os
import urllib.request
import numpy as np
import MDAnalysis as mda
import py3Dmol
import pytraj as pt
import platform
import scipy.cluster.hierarchy
from scipy.spatial.distance import squareform
import scipy.stats as stats
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import griddata
import seaborn as sb
from statistics import mean, stdev
from pytraj import matrix
from matplotlib import colors
from IPython.display import set_matplotlib_formats
import subprocess

## **Using Google Drive to store simulation data**

Google Colab does not allow users to keep data on their computer hard drives. However, we can use Google Drive to read, write, and store our simulations files.

To do this, create a folder in your own Google Drive and copy the necessary input file there (.pdb file). **Make sure there are no spaces or capitalizations in the folder name or the name of your .pdb file!**


In [ ]:
#@title ### **1c. Import Google Drive**
#@markdown Click PLAY to make your Google Drive accessible.
from google.colab import drive

drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

---
---
# **Loading your .pdb file**

At this point, you should have all libraries and dependencies installed and all necessary input files already in your Google Drive folder. This is what you did in the last section! Now, you will upload the file that describes the structure of your protein. This file is called a .pdb file.

Below, you should provide the names of all input files and the pathway of your Google Drive folder containing them.

*   **PDB_file_name** is where you will put the name of your .pdb file. Add the name where it says "xxx". Make sure there are no spaces or capitalizations in the file name!
*  **Google_Drive_path** is where you will put the name of the google drive folder you want to store your data. After MyDrive/, paste the name of your folder where the "xxx" is. Make sure there are no spaces in the folder name! Also, make sure the PDB file points to the correct pathway. If necessary, correct the pathway and re-upload the files.
* **remove_waters** box should say yes. You do not need to edit this unless your instructor says otherwise.

When you are ready, press PLAY.

**If this code has run successfully, you should see new files in the folder you made!**


In [ ]:
#@title 2. Please provide the necessary input file below:
#@markdown Please reference the text box above for instructions regarding file name inputs and directories.
%%capture
PDB_file_name = 'xxx.pdb' #@param {type:"string"}
file_name = PDB_file_name
Google_Drive_Path = '/content/drive/MyDrive/xxx' #@param {type:"string"}
workDir = Google_Drive_Path

remove_waters = "yes" #@param ["yes", "no" ]
if remove_waters == "yes":
  no_waters = "nowat"
else:
  no_waters = ''

initial_pdb = os.path.join(workDir, str(file_name))
starting = os.path.join(workDir, "starting.pdb")
starting2 = os.path.join(workDir, "starting2.pdb")
starting3 = os.path.join(workDir, "starting3.pdb")
prepareforleap = os.path.join(workDir, "prepareforleap.in")
starting_end = os.path.join(workDir, "starting_end.pdb")
tleap = os.path.join(workDir, "tleap.in")
top_nw = os.path.join(workDir, "SYS_nw.prmtop")
crd_nw = os.path.join(workDir, "SYS_nw.crd")
pdb_nw = os.path.join(workDir, "SYS_nw.pdb")
top = os.path.join(workDir, "SYS.prmtop")
crd = os.path.join(workDir, "SYS.crd")
pdb = os.path.join(workDir, "SYS.pdb")

ppdb = PandasPdb().read_pdb(initial_pdb)
ppdb.df['ATOM'] = ppdb.df['ATOM']
# ppdb.df['HETATM'] = ppdb.df['HETATM'][ppdb.df['HETATM']['residue_name'] == 'HOH']
ppdb.df['ATOM'] = ppdb.df['ATOM'][ppdb.df['ATOM']['atom_name'] != 'OXT']
ppdb.df['ATOM']= ppdb.df['ATOM'][ppdb.df['ATOM']['element_symbol'] != 'H']
ppdb.to_pdb(path=starting, records=['ATOM', 'HETATM'], gz=False, append_newline=True)

# from Bio.PDB import is_aa
# from Bio.PDB import PDBParser, PDBIO, Select


# class ProtSelect(Select):
#     def accept_residue(self, residue):
#         print(f"{residue} -> {is_aa(residue)}")
#         return is_aa(residue, standard=True)


# from Bio import PDB

# pdb_ini = PDBParser().get_structure("pdb", starting)
# io = PDBIO()
# io.set_structure(pdb_ini)
# io.save(starting2, ProtSelect());



# pdb4amber_cmd = "pdb4amber -i " + str(starting2) + " -o " + str(starting_end) + " -p"

# original_stdout = sys.stdout # Save a reference to the original standard output

# with open('pdb4amber.sh', 'w') as f:
#     sys.stdout = f # Change the standard output to the file we created.
#     print(pdb4amber_cmd)
#     sys.stdout = original_stdout # Reset the standard output to its original value

# !chmod 700 pdb4amber.sh 2>&1 1>/dev/null
# !bash pdb4amber.sh 2> /dev/null

def remove_lines(filename):
    with open(filename, 'r') as file:
        ter_count = 0
        for line in file:
            if line.startswith('TER'):
                ter_count += 1
                if ter_count >= 1:
                    yield line
                    for i in range(3):
                        line = next(file, None)
                        if line is not None and line.startswith('ATOM') and line.split()[2] in ['P', 'OP1', 'OP2']:
                            continue
                        else:
                            yield line
                else:
                    yield line
            else:
                yield line


f = open(prepareforleap, "w")
f.write("""parm """ + str(starting) + "\n"
"""loadcrd """ + str(starting) + """ name edited""" + "\n"
"""prepareforleap crdset edited name from-prepareforleap \ """ + "\n"
"""pdbout """ + str(starting2) + " " + str(no_waters) + """ noh""" + "\n"
"""go """)
f.close()

prepareforleap_command = "cpptraj -i " + str(prepareforleap)
original_stdout = sys.stdout # Save a reference to the original standard output
with open('prepareforleap.sh', 'w') as f:
    sys.stdout = f # Change the standard output to the file we created.
    print(prepareforleap_command)
    sys.stdout = original_stdout # Reset the standard output to its original value

subprocess.run(["chmod 700 prepareforleap.sh"], shell=True)
subprocess.run(["./prepareforleap.sh"], shell=True,)


pdb4amber_cmd = "pdb4amber -i " + str(starting2) + " -o " + str(starting3) + " -a"
original_stdout = sys.stdout # Save a reference to the original standard output

with open('pdb4amber.sh', 'w') as f:
    sys.stdout = f # Change the standard output to the file we created.
    print(pdb4amber_cmd)
    sys.stdout = original_stdout # Reset the standard output to its original value

subprocess.run(["chmod 700 pdb4amber.sh"], shell=True)
subprocess.run(["./pdb4amber.sh"], shell=True,)

with open(starting_end, 'w') as out_file:
    for line in remove_lines(starting3):
        if line is not None:
          out_file.write(line)
        else:
          pass



#@markdown ---

---
# **Generating Topology and Defining Your Universe**

By now, the code has recognized your .pdb file and the directory assigned for that file. Now, the program will analyze the topology, which is a complete description of how your system will behave. Topology gives details including residues, atom names, atom types, charges, connections, masses, bonded parameters, and nonbonded parameters. In simpler terms, this will define your "universe", which is a box of space in the universe that is under your control. All of the parameters that you use will define this universe that surrounds your protein. You will see this universe as a bubbly box around your protein when the code has finished running.

For more information, reference section 3.1 in the AMBER 2023 manual: https://ambermd.org/doc12/Amber23.pdf. We strongly recommend using ff19SB with TIP3P.

*   **Force_field** explains to the simulation how the protein is supposed to function in a specific force field. Use ff19SB, as ff14SB are the old parameters, while ff19SB are the new ones.
*   **Water_type** describes how many center points are on your water molecule. Use TIP3P to create three center points on your molecule.
* **Size_box**: Keep this at 12.
* **Ions** is the type of ion you would like to use to neutralize your protein. This  will depend on your protein.
* **Concentration** is the concentration of ion that you would like to use to neutralize your protein. This  will depend on your protein.

For ions and concentration, we recommend using 100mM NaCl. However, this can be adjusted for what is recommeded for your specific protein.

**If this code has run successfully, you should see new files in the folder you made!**






In [ ]:
#@title **3. Parameters to generate the Amber topology:**
#@markdown This may take a while to run, so please be patient!
Force_field = "ff19SB" #@param ["ff19SB", "ff14SB"]
if Force_field == "ff19SB":
  ff = "leaprc.protein.ff19SB"
else:
  ff = "leaprc.protein.ff14SB"

Water_type = "TIP3P" #@param ["TIP3P", "OPC"]
if Water_type == "TIP3P":
  water = "leaprc.water.tip3p"
  water_box = "TIP3PBOX"
else:
  water = "leaprc.water.opc"
  water_box = "OPCBOX"

#@markdown Size Box (Angstrons):

Size_box = 12 #@param {type:"slider", min:10, max:20, step:1}
size_box = Size_box

#@markdown **ATTENTION**: Give the concentration in Molar units, AMBER tleap will neutralize your system automatically:

Ions = "NaCl" #@param ["NaCl", "KCl" ]

Concentration = "0.10" #@param {type:"string"}

#@markdown ---

f = open(tleap, "w")
f.write("""source """ + str(ff) + "\n"
"""source leaprc.DNA.OL15
source leaprc.RNA.OL3
source leaprc.GLYCAM_06j-1
source leaprc.gaff2
source """  + str(water) + "\n"
"""SYS = loadpdb """  + str(starting_end) + "\n"
"""alignaxes SYS
savepdb SYS """ + str(pdb_nw) + "\n"
"""saveamberparm SYS """ + str(top_nw) + " " + str(crd_nw) + "\n"
"""solvatebox SYS """ + str(water_box) + " " + str(size_box) +  """ 0.7
saveamberparm SYS """ + str(top) + " " + str(crd) + "\n"
"""savepdb SYS """ + str(pdb) + "\n"
"""quit""")
f.close()

tleap_command = "tleap -f " + str(tleap)

original_stdout = sys.stdout # Save a reference to the original standard output

with open('run_tleap.sh', 'w') as f:
    sys.stdout = f # Change the standard output to the file we created.
    print(tleap_command)
    sys.stdout = original_stdout # Reset the standard output to its original value

SYS = os.path.join(workDir, "SYS*")
rm_sys = "rm " + SYS

original_stdout = sys.stdout # Save a reference to the original standard output

with open('rm_sys.sh', 'w') as f:
    sys.stdout = f # Change the standard output to the file we created.
    print(rm_sys)
    sys.stdout = original_stdout # Reset the standard output to its original value

subprocess.run(["chmod 700 rm_sys.sh"], shell=True)
subprocess.run(["./rm_sys.sh"], shell=True,)
subprocess.run(["chmod 700 run_tleap.sh"], shell=True)
subprocess.run(["./run_tleap.sh"], shell=True,)


subprocess.run(['grep "Volume:" leap.log > temp.txt'], shell=True)
with open("temp.txt", 'r') as f:
  for line in f:
        vol = float(line.split()[1])

vol_lit  = vol * pow(10, -27)
atom_lit = 9.03 * pow(10, 22)
conc = float(Concentration)
num_ion = int(vol_lit * (conc/0.15) * atom_lit)

if Ions == "NaCl":
  pos_neut = "Na+ 0"
  pos_num = "Na+ " + str(num_ion)
  Cl_num = num_ion
else:
  pos_neut = "K+ 0"
  pos_num = "K+ " + str(num_ion)
  Cl_num = num_ion

f = open(tleap, "w")
f.write("""source """ + str(ff) + "\n"
"""source leaprc.DNA.OL15
source leaprc.RNA.OL3
source leaprc.GLYCAM_06j-1
source leaprc.gaff2
source """  + str(water) + "\n"
"""SYS = loadpdb """  + str(starting_end) + "\n"
"""alignaxes SYS
check SYS
charge SYS
addions SYS """ + str(pos_neut) + "\n"
"""addions SYS Cl- 0
check SYS
charge SYS
savepdb SYS """ + str(pdb_nw) + "\n"
"""saveamberparm SYS """ + str(top_nw) + " " + str(crd_nw) + "\n"
"""solvatebox SYS """ + str(water_box) + " " + str(size_box) +  """ 0.7 """ + "\n"
"""addIonsRand SYS """ + str(pos_num) + """ Cl- """ + str(Cl_num) + "\n"
"""saveamberparm SYS """ + str(top) + " " + str(crd) + "\n"
"""savepdb SYS """ + str(pdb) + "\n"
"""quit""")
f.close()

subprocess.run(["chmod 700 run_tleap.sh"], shell=True)
subprocess.run(["./run_tleap.sh"], shell=True,)

pdb_amber = os.path.exists(pdb)
top_amber = os.path.exists(top)
crd_amber = os.path.exists(crd)

subprocess.run(["rm *.sh temp.txt"], shell=True,)

if pdb_amber == True and top_amber == True and crd_amber == True:
  print("Successfully generated topology! :-)")
else:
  print("ERROR: Check your input file! ")

---
## Building a simulation box:

This is how you will build your 3D simulation. You do not need to edit anything, as these are all aesthetic preference settings.

In [ ]:
#@title **4. Show 3D structure**
import warnings
warnings.filterwarnings('ignore')
import py3Dmol

color = "gray" #@param ["gray", "rainbow"]
show_sidechains = False #@param {type:"boolean"}
show_mainchains = False #@param {type:"boolean"}
show_box = True #@param {type:"boolean"}
box_opacity = 0.6 #@param {type:"slider", min:0, max:1, step:0.1}


def show_pdb(show_sidechains=False, show_mainchains=False, show_box = False, color="rainbow"):
  view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',)
  view.addModel(open(pdb,'r').read(),'pdb')

  if color == "gray":
    view.setStyle({'cartoon':{}})
  elif color == "rainbow":
    view.setStyle({'cartoon': {'color':'spectrum'}})

  if show_sidechains:
    BB = ['C','O','N']
    view.addStyle({'and':[{'resn':["GLY","PRO"],'invert':True},{'atom':BB,'invert':True}]},
                        {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"GLY"},{'atom':'CA'}]},
                        {'sphere':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"PRO"},{'atom':['C','O'],'invert':True}]},
                        {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
  if show_mainchains:
    BB = ['C','O','N','CA']
    view.addStyle({'atom':BB},{'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})

  if show_box:
    view.addSurface(py3Dmol.SAS, {'opacity': box_opacity, 'color':'white'})

  view.zoomTo()
  return view


show_pdb(show_sidechains, show_mainchains, show_box, color).show()

---
---
# **Equilibrating the simulation box**

Proper MD equilibration protocol is designed to equilibrate both temperature and pressure throughout the simulation box while preserving the protein experimental conformation. In simple terms, an MD equilibration step is used to bring your protein to the  temperature and pressure you want for analysis before you start to run all of the code that takes a long time. In addition, we also allow the solvent to accomodate around the protein, creating proper solvation layers. This is the main step we use to fix any issues that the code didn't see before.

Below, you will set up the MD equilibration parameters, such as temperature, pressure and the desired simulation time. We will define the force constant used to restrain protein heavy-atoms in place and the frequency at which we want to save atomic coordinates in a trajectory file (.dcd).

.dcd is a file format that store simulation trajectories. Simulation trajectories are a dynamic view of various properties in your protein, including atom coordinates, box size, velocities, and forces. Trajectory files looks at these variables as they change throughout time.



*   **Jobname**: enter the name you would like to give your job where it says "xxx". Do not include any spaces!
*   **Minimization Steps**: These steps optomize the atoms in your protein to reduce the net forces of these atoms until they become negligible. In simple terms, it finds the stable state of your protein, which helps optomize your molecular structure.
* **Simulation time**: This is how long you would like to run the simulation in nanoseconds.
* **Integration_timestep**: The timestep holds your data that varies with time (dynamic data), such as coordinates, velocities, and energies, for a single step in a trajectory. Choosing too small of a timestep leads to an unrealistic simulation time, whereas too big of a timestep leads to the system not being represented correctly.
* **Temperature**: Enter the desired temperature you would like your protein to be at in Kelvin. Standard temperature is 298K, which is the default. Only change these values if your professor has explicitly told you to. If you change the value, do not include units, only type the number.
* **Pressure** : Enter the desired pressure you would like your protein to be at in bar. Standard pressure is 1 bar, which is the default. Only change these values if your professor has explicitly told you to. If you change the value, do not include units, only type the number.
* **Force_constant**: Keep this at 500.
* **Write_the_trajectory**: The trajectory files will give you small snapshots of your protein throguhout the MD equilibration. Include enough snapshots so that you can view changes in your protein, but not too many that you obtain an enormous amount of data files. Ask your instructor for the specific trajectory value to imput.
* **Write_the_log**: This relates to how many log files your computer will save during this run. Keep this at 10 unless your instructor says otherwise. 10 = one log file every 10 picoseconds.



**This run should take about 5-10 minutes, so please sit back and relax!**

After you are done, you can run the next 2 cells to equilibrate your system.

In [ ]:
#@title ### **5a. Parameters for MD Equilibration protocol:**

# remove whitespaces
Jobname = 'xxx_equil' #@param {type:"string"}

Minimization_steps = "1000" #@param ["1000", "5000", "10000", "20000", "50000", "100000"]

#@markdown Simulation time (in nanoseconds) and integration time (in femtoseconds):
Time = "0.2" #@param {type:"string"}
stride_time_eq = Time
Integration_timestep = "2" #@param ["0.5", "1", "2", "3", "4"]
dt_eq = Integration_timestep

#@markdown Temperature (in Kelvin) and Pressure (in bar)
Temperature = 298 #@param {type:"string"}
temperature_eq = Temperature
Pressure = 1 #@param {type:"string"}
pressure_eq = Pressure

#@markdown Position restraints force constant (in kJ/mol):
Force_constant = 500 #@param {type:"slider", min:0, max:2000, step:100}

#@markdown Frequency to write the trajectory file (in picoseconds):

Write_the_trajectory = "100" #@param ["10", "100", "200", "500", "1000"]
write_the_trajectory_eq = Write_the_trajectory
#@markdown Frequency to write the log file (in picoseconds):

Write_the_log = "10" #@param ["10", "100", "200", "500", "1000"]
write_the_log_eq = Write_the_log


#@markdown ---


In [ ]:
#@title **5b. Runs an Equilibration MD simulation (NPT ensemble)**
#@markdown Now, let's equilibrate our system! All you need to do is press PLAY.

###########################################
import openmm as mm
from openmm import *
from openmm.app import *
from openmm.unit import *
import pytraj as pt

from sys import stdout, exit, stderr
import os, math, fnmatch

#############################################
# Defining MD simulation parameters

jobname = os.path.join(workDir, Jobname)
coordinatefile = os.path.join(workDir, "SYS.crd")
pdbfile = os.path.join(workDir, "SYS.pdb")
topologyfile = os.path.join(workDir, "SYS.prmtop")

time_ps = float(Time)*1000
simulation_time = float(time_ps)*picosecond		# in ps
dt = int(dt_eq)*femtosecond
temperature = float(temperature_eq)*kelvin
savcrd_freq = int(write_the_trajectory_eq)*picosecond
print_freq  = int(write_the_log_eq)*picosecond

pressure	= float(pressure_eq)*bar

restraint_fc = int(Force_constant) # kJ/mol

nsteps  = int(simulation_time.value_in_unit(picosecond)/dt.value_in_unit(picosecond))
nprint  = int(print_freq.value_in_unit(picosecond)/dt.value_in_unit(picosecond))
nsavcrd = int(savcrd_freq.value_in_unit(picosecond)/dt.value_in_unit(picosecond))

#############################################
# Defining functions to use below:
def backup_old_log(pattern, string):
	result = []
	for root, dirs, files in os.walk("./"):
		for name in files:
			if fnmatch.fnmatch(name, pattern):

				try:
					number = int(name[-2])
					avail = isinstance(number, int)
					#print(name,avail)
					if avail == True:
						result.append(number)
				except:
					pass

	if len(result) > 0:
		maxnumber = max(result)
	else:
		maxnumber = 0

	backup_file = "\#" + string + "." + str(maxnumber + 1) + "#"
	os.system("mv " + string + " " + backup_file)
	return backup_file

def restraints(system, crd, fc, restraint_array):

	boxlx = system.getDefaultPeriodicBoxVectors()[0][0].value_in_unit(nanometers)
	boxly = system.getDefaultPeriodicBoxVectors()[1][1].value_in_unit(nanometers)
	boxlz = system.getDefaultPeriodicBoxVectors()[2][2].value_in_unit(nanometers)

	if fc > 0:
		# positional restraints for all heavy-atoms
		posresPROT = CustomExternalForce('k*periodicdistance(x, y, z, x0, y0, z0)^2;')
		posresPROT.addPerParticleParameter('k')
		posresPROT.addPerParticleParameter('x0')
		posresPROT.addPerParticleParameter('y0')
		posresPROT.addPerParticleParameter('z0')

		for atom1 in restraint_array:
			atom1 = int(atom1)

			xpos  = crd.positions[atom1].value_in_unit(nanometers)[0]
			ypos  = crd.positions[atom1].value_in_unit(nanometers)[1]
			zpos  = crd.positions[atom1].value_in_unit(nanometers)[2]

			posresPROT.addParticle(atom1, [fc, xpos, ypos, zpos])

		system.addForce(posresPROT)

	return system
##############################################

#############################################
print("\n> Simulation details:\n")
print("\tJob name = " + jobname)
print("\tCoordinate file = " + str(coordinatefile))
print("\tPDB file = " + str(pdbfile))
print("\tTopology file = " + str(topologyfile))

print("\n\tSimulation_time = " + str(simulation_time))
print("\tIntegration timestep = " + str(dt))
print("\tTotal number of steps = " +  str(nsteps))

print("\n\tSave coordinates each " + str(savcrd_freq))
print("\tPrint in log file each " + str(print_freq))

print("\n\tTemperature = " + str(temperature))
print("\tPressure = " + str(pressure))
#############################################

print("\n> Setting the system:\n")

print("\t- Reading topology and structure file...")
prmtop = AmberPrmtopFile(topologyfile)
inpcrd = AmberInpcrdFile(coordinatefile)

print("\t- Creating system and setting parameters...")
nonbondedMethod = PME
nonbondedCutoff = 1.0*nanometers
ewaldErrorTolerance = 0.0005
constraints = HBonds
rigidWater = True
constraintTolerance = 0.000001
friction = 1.0
system = prmtop.createSystem(nonbondedMethod=nonbondedMethod, nonbondedCutoff=nonbondedCutoff,
                           constraints=constraints, rigidWater=rigidWater, ewaldErrorTolerance=ewaldErrorTolerance)

print("\t- Applying restraints. Force Constant = " + str(Force_constant) + "kJ/mol")
pt_system = pt.iterload(coordinatefile, topologyfile)
pt_topology = pt_system.top
restraint_array = pt.select_atoms('!(:H*) & !(:WAT) & !(:Na+) & !(:Cl-) & !(:Mg+) & !(:K+)', pt_topology)

system = restraints(system, inpcrd, restraint_fc, restraint_array)

print("\t- Setting barostat...")
system.addForce(MonteCarloBarostat(pressure, temperature))

print("\t- Setting integrator...")
integrator = LangevinIntegrator(temperature, friction, dt)
integrator.setConstraintTolerance(constraintTolerance)
simulation = Simulation(prmtop.topology, system, integrator)
simulation.context.setPositions(inpcrd.positions)
if inpcrd.boxVectors is not None:
    simulation.context.setPeriodicBoxVectors(*inpcrd.boxVectors)

print("\t- Energy minimization: " + str(Minimization_steps) + " steps")
simulation.minimizeEnergy(tolerance=10*kilojoule/mole, maxIterations=int(Minimization_steps))
print("\t-> Potential Energy = " + str(simulation.context.getState(getEnergy=True).getPotentialEnergy()))

print("\t- Setting initial velocities...")
simulation.context.setVelocitiesToTemperature(temperature)

#############################################
# Running Equilibration on NPT ensemble

dcd_file = jobname + ".dcd"
log_file = jobname + ".log"
rst_file = jobname + ".rst"
prv_rst_file = jobname + ".rst"
pdb_file = jobname + ".pdb"

# Creating a trajectory file and reporters
dcd = DCDReporter(dcd_file, nsavcrd)
firstdcdstep = (nsteps) + nsavcrd
dcd._dcd = DCDFile(dcd._out, simulation.topology, simulation.integrator.getStepSize(), firstdcdstep, nsavcrd) # charmm doesn't like first step to be 0

simulation.reporters.append(dcd)
simulation.reporters.append(StateDataReporter(stdout, nprint, step=True, speed=True, progress=True, totalSteps=nsteps, remainingTime=True, separator='\t\t'))
simulation.reporters.append(StateDataReporter(log_file, nprint, step=True, kineticEnergy=True, potentialEnergy=True, totalEnergy=True, temperature=True, volume=True, speed=True))

print("\n> Simulating " + str(nsteps) + " steps...")
simulation.step(nsteps)

simulation.reporters.clear() # remove all reporters so the next iteration don't trigger them.


##################################
# Writing last frame information of stride
print("\n> Writing state file (" + str(rst_file) + ")...")
state = simulation.context.getState( getPositions=True, getVelocities=True )
with open(rst_file, 'w') as f:
	f.write(XmlSerializer.serialize(state))

last_frame = int(nsteps/nsavcrd)
print("> Writing coordinate file (" + str(pdb_file) + ", frame = " + str(last_frame) + ")...")
positions = simulation.context.getState(getPositions=True).getPositions()
PDBFile.writeFile(simulation.topology, positions, open(pdb_file, 'w'))

print("\n> Finished!\n")

---
---
# **Running a Production MD simulation**

Finally, we will proceed with the Production simulation itself using the equilibrated system coordinates as input structure.

Note that we will use here a *.rst state file* , which contains atomic velocities and positions from the last frame of the equilibration simulation, guaranteeing that our production simulation begins from a thermodynamically equilibrated system.

Another important information here is the **Number_of_strides** and the **Stride_Time**. In this notebook, we simulate a defined number of *strides*, so the **simulation time = Number_of_strides*Stride_Time**. For example, we can simulate 100ns by setting *Number_of_strides=10* and *Stride_Time=10 ns*.

**Important: at the end of the Production simulation, we concatenate all strides to create a complete trajectory file which can be visualized and analyzed**



*   **Equilibrated_PDB**: Insert the name of the newly equilibrated .pdb file. This will be the same name that you designated earlier in "Jobname".
*   **State_file**: This is where you will insert the name of your state file. This will be the same name that you designated earlier in "Jobname".
* **Jobname**: enter the name you would like to give your job. Do not include any spaces!
* **Stride_Time**: Enter your desired stride time, as described above. Ask your instructor for a desired simulation time to determine this value.
* **Number_of_strides**: Enter your desired number of strides, as described above. Ask your instructor for a desired simulation time to determine this value.
* **Integration_timestep**: The timestep holds your data that varies with time (dynamic data), such as coordinates, velocities, and energies, for a single step in a trajectory. Choosing too small of a timestep leads to an unrealistic simulation time, whereas too big of a timestep leads to the system not being represented correctly.
* **Temperature**: Enter the desired temperature you would like your protein to be at in Kelvin. Standard temperature is 298K, which is the default. Only change these values if your professor has explicitly told you to. If you change the value, do not include units, only type the number.
* **Pressure** : Enter the desired pressure you would like your protein to be at in bar. Standard pressure is 1 bar, which is the default. Only change these values if your professor has explicitly told you to. If you change the value, do not include units, only type the number.
* **Force_constant**: Keep this at 500.
* **Write_the_trajectory**: The trajectory files will give you small snapshots of your protein throguhout the MD equilibration. Include enough snapshots so that you can view changes in your protein, but not too many that you obtain an enormous amount of data files. Ask your instructor for the specific trajectory value to imput.
* **Write_the_log**: This relates to how many log files your computer will save during this run. Keep this at 10 unless your instructor says otherwise. 10 = one log file every 10 picoseconds.



In [ ]:
#@markdown ### **6a. Provide input file names below:**

Equilibrated_PDB = 'xxx.pdb' #@param {type:"string"}
State_file = 'xxx.rst' #@param {type:"string"}

#@markdown ---
#@markdown ### **Parameters for MD Production protocol:**


# remove whitespaces
Jobname = 'xxx' #@param {type:"string"}

#@markdown Simulation time (in nanoseconds), number of strides (integers) and integration timestep (in femtoseconds):
Stride_Time = "2" #@param {type:"string"}
stride_time_prod = Stride_Time
Number_of_strides = "1" #@param {type:"string"}
nstride = Number_of_strides
Integration_timestep = "3" #@param ["0.5", "1", "2", "3", "4"]
dt_prod = Integration_timestep

#@markdown Temperature (in Kelvin) and Pressure (in bar)
Temperature = 298 #@param {type:"string"}
temperature_prod = Temperature
Pressure = 1 #@param {type:"string"}
pressure_prod = Pressure

#@markdown Frequency to write the trajectory file (in picoseconds):
Write_the_trajectory = "10" #@param ["10", "100", "200", "500", "1000"]
write_the_trajectory_prod = Write_the_trajectory

#@markdown Frequency to write the log file (in picoseconds):
Write_the_log = "10" #@param ["10", "100", "200", "500", "1000"]
write_the_log_prod = Write_the_log

#@markdown ---

In [ ]:
#@title **6b. Runs a Production MD simulation (NPT ensemble) after equilibration**
#
###########################################
import openmm as mm
from openmm import *
from openmm.app import *
from openmm.unit import *

from sys import stdout, exit, stderr
import os, math, fnmatch

#############################################
# Defining MD simulation parameters

jobname = os.path.join(workDir, str(Jobname))
coordinatefile = os.path.join(workDir, "SYS.crd")
pdbfile = os.path.join(workDir, Equilibrated_PDB)
topologyfile = os.path.join(workDir, "SYS.prmtop")
equil_rst_file = os.path.join(workDir, State_file)


stride_time_ps = float(stride_time_prod)*1000
stride_time = float(stride_time_ps)*picosecond
nstride = int(Number_of_strides)
dt = int(dt_prod)*femtosecond
temperature = float(temperature_prod)*kelvin
savcrd_freq = int(write_the_trajectory_prod)*picosecond
print_freq  = int(write_the_log_prod)*picosecond

pressure	= float(pressure_prod)*bar

simulation_time = stride_time*nstride
nsteps  = int(stride_time.value_in_unit(picosecond)/dt.value_in_unit(picosecond))
nprint  = int(print_freq.value_in_unit(picosecond)/dt.value_in_unit(picosecond))
nsavcrd = int(savcrd_freq.value_in_unit(picosecond)/dt.value_in_unit(picosecond))
firststride = 1 # must be integer
#############################################
# Defining functions to use below:
def backup_old_log(pattern, string):
	result = []
	for root, dirs, files in os.walk("./"):
		for name in files:
			if fnmatch.fnmatch(name, pattern):

				try:
					number = int(name[-2])
					avail = isinstance(number, int)
					#print(name,avail)
					if avail == True:
						result.append(number)
				except:
					pass

	if len(result) > 0:
		maxnumber = max(result)
	else:
		maxnumber = 0

	backup_file = "\#" + string + "." + str(maxnumber + 1) + "#"
	os.system("mv " + string + " " + backup_file)
	return backup_file
##############################################

#############################################
print("\n> Simulation details:\n")
print("\tJob name = " + jobname)
print("\tCoordinate file = " + str(coordinatefile))
print("\tPDB file = " + str(pdbfile))
print("\tTopology file = " + str(topologyfile))

print("\n\tSimulation_time = " + str(stride_time*nstride))
print("\tIntegration timestep = " + str(dt))
print("\tTotal number of steps = " +  str(nsteps*nstride))
print("\tNumber of strides = " + str(nstride) + " (" + str(stride_time) + " in each stride)")

print("\n\tSave coordinates each " + str(savcrd_freq))
print("\tSave checkpoint each " + str(savcrd_freq))
print("\tPrint in log file each " + str(print_freq))

print("\n\tTemperature = " + str(temperature))
print("\tPressure = " + str(pressure))
#############################################

print("\n> Setting the system:\n")

print("\t- Reading topology and structure file...")
prmtop = AmberPrmtopFile(topologyfile)
inpcrd = AmberInpcrdFile(coordinatefile)


print("\t- Creating system and setting parameters...")
nonbondedMethod = PME
nonbondedCutoff = 1.0*nanometers
ewaldErrorTolerance = 0.0005
constraints = HBonds
rigidWater = True
constraintTolerance = 0.000001
friction = 1.0
system = prmtop.createSystem(nonbondedMethod=nonbondedMethod, nonbondedCutoff=nonbondedCutoff,
                           constraints=constraints, rigidWater=rigidWater, ewaldErrorTolerance=ewaldErrorTolerance)

print("\t- Setting barostat...")
system.addForce(MonteCarloBarostat(pressure, temperature))

print("\t- Setting integrator...")
integrator = LangevinIntegrator(temperature, friction, dt)
integrator.setConstraintTolerance(constraintTolerance)
simulation = Simulation(prmtop.topology, system, integrator)
simulation.context.setPositions(inpcrd.positions)
if inpcrd.boxVectors is not None:
	simulation.context.setPeriodicBoxVectors(*inpcrd.boxVectors)

#############################################
# Opening a loop of extension NSTRIDE to simulate the entire STRIDE_TIME*NSTRIDE
for n in range(1, nstride + 1):

	print("\n\n>>> Simulating Stride #" + str(n) + " <<<")

	dcd_file = jobname + "_" + str(n) + ".dcd"
	log_file = jobname + "_" + str(n) + ".log"
	rst_file = jobname + "_" + str(n) + ".rst"
	prv_rst_file = jobname + "_" + str(n-1) + ".rst"
	pdb_file = jobname + "_" + str(n) + ".pdb"

	if os.path.exists(rst_file):
		print("> Stride #" + str(n) + " finished (" + rst_file + " present). Moving to next stride... <")
		continue

	if n == 1:
		print("\n> Loading previous state from equilibration > " + equil_rst_file + " <")
		with open(equil_rst_file, 'r') as f:
			simulation.context.setState(XmlSerializer.deserialize(f.read()))
			currstep = int((n-1)*nsteps)
			currtime = currstep*dt.in_units_of(picosecond)
			simulation.currentStep = currstep
			simulation.context.setTime(currtime)
			print("> Current time: " + str(currtime) + " (Step = " + str(currstep) + ")")

	else:
		print("> Loading previous state from > " + prv_rst_file + " <")
		with open(prv_rst_file, 'r') as f:
			simulation.context.setState(XmlSerializer.deserialize(f.read()))
			currstep = int((n-1)*nsteps)
			currtime = currstep*dt.in_units_of(picosecond)
			simulation.currentStep = currstep
			simulation.context.setTime(currtime)
			print("> Current time: " + str(currtime) + " (Step = " + str(currstep) + ")")


	dcd = DCDReporter(dcd_file, nsavcrd)
	firstdcdstep = (currstep) + nsavcrd
	dcd._dcd = DCDFile(dcd._out, simulation.topology, simulation.integrator.getStepSize(), firstdcdstep, nsavcrd) # first step should not be 0

	simulation.reporters.append(dcd)
	simulation.reporters.append(StateDataReporter(stdout, nprint, step=True, speed=True, progress=True, totalSteps=(nsteps*nstride), remainingTime=True, separator='\t\t'))
	simulation.reporters.append(StateDataReporter(log_file, nprint, step=True, kineticEnergy=True, potentialEnergy=True, totalEnergy=True, temperature=True, volume=True, speed=True))

	print("\n> Simulating " + str(nsteps) + " steps... (Stride #" + str(n) + ")")
	simulation.step(nsteps)

	simulation.reporters.clear() # remove all reporters so the next iteration don't trigger them.


	##################################
	# Writing last frame information of stride
	print("\n> Writing state file (" + str(rst_file) + ")...")
	state = simulation.context.getState( getPositions=True, getVelocities=True )
	with open(rst_file, 'w') as f:
		f.write(XmlSerializer.serialize(state))

	last_frame = int(nsteps/nsavcrd)
	print("> Writing coordinate file (" + str(pdb_file) + ", frame = " + str(last_frame) + ")...")
	positions = simulation.context.getState(getPositions=True).getPositions()
	PDBFile.writeFile(simulation.topology, positions, open(pdb_file, 'w'))

print("\n> Finished!\n")

In [ ]:
#@title **7. Concatenate and align the trajectory**
#@markdown **Important**: The **Google Drive Path**, **Jobname**, **Number of strides**, **stride time** and **trajectory saved frequency** should be the same you have been used to run your simulation in the previous steps.

import MDAnalysis as mda
from MDAnalysis.analysis import align, rms

Google_Drive_Path = '/content/drive/MyDrive/xxx' #@param {type:"string"}
workDir = Google_Drive_Path
Equilibrated_PDB = 'xxx.pdb' #@param {type:"string"}
Jobname = "xxx" #@param {type: "string"}
Skip = "1" #@param ["1", "2", "5", "10", "20", "50"]
stride_traj = Skip
Output_format = "dcd" #@param ["dcd", "pdb", "trr", "xtc"]
first_stride = "1" #@param {type:"string"}
Number_of_strides = "1" #@param {type:"string"}
nstride = int(Number_of_strides)
stride_time = "2" #@param {type:"string"}
trajectory_saved_frequency = "10" #@param ["10", "100", "200", "500", "1000"]
traj_save_freq = trajectory_saved_frequency
Remove_waters = "yes" #@param ["yes", "no"]
# stride_id_as_ref_for_alignment = "1" #@param {type: "string"}
output_prefix = first_stride+"-"+str(int(first_stride)+nstride-1)

stride_time_ps = float(stride_time)*1000
simulation_time_analysis = stride_time_ps*nstride
simulation_ns = float(stride_time)*int(Number_of_strides)
number_frames = int(simulation_time_analysis)/int(traj_save_freq)
number_frames_analysis = number_frames/int(stride_traj)


# traj_end = os.path.join(workDir, str(Jobname) + "_all.dcd")
nw_dcd = os.path.join(workDir, str(Jobname) + output_prefix + "_nw." + str(Output_format))
nw_pdb = os.path.join(workDir, str(Jobname) +  "_nw.pdb")
whole_pdb = os.path.join(workDir, str(Jobname) +  "_whole.pdb")
whole_dcd = os.path.join(workDir, str(Jobname) + output_prefix + "_whole." + str(Output_format))
template =  os.path.join(workDir, str(Jobname) + '_%s.dcd')
pdb = os.path.join(workDir, Equilibrated_PDB)

flist = [template % str(i) for i in range(int(first_stride), int(first_stride) + nstride)]
ref = [template % int(1)]

u1 = mda.Universe(pdb, flist)
u2 = mda.Universe(pdb, ref)

u2.trajectory[0] # set u2 to first frame

# print(rms.rmsd(u1.select_atoms('name CA').positions, u2.select_atoms('name CA').positions, superposition=False))

align.AlignTraj(u1, u2, select='name CA', in_memory=True).run()

nw = u1.select_atoms("not (resname HOH)")
if Remove_waters == "yes":
  with mda.Writer(nw_dcd, nw.n_atoms) as W:
    for ts in u1.trajectory[::int(Skip)]:
        W.write(nw, )
  not_waters = u2.select_atoms("not (resname HOH)")
  not_waters.write(nw_pdb)
  traj_dcd_check = os.path.exists(nw_dcd)
  traj = nw_dcd
  pdb_ref = nw_pdb
else:
  with mda.Writer(whole_dcd, u1.select_atoms("all").n_atoms) as W:
    for ts in u1.trajectory[::int(Skip)]:
        W.write(u1.select_atoms("all"))
  whole = u2.select_atoms("all")
  whole.write(whole_pdb)
  traj_dcd_check = os.path.exists(whole_dcd)
  traj = whole_dcd
  pdb_ref = whole_pdb

traj_load = pt.load(traj, pdb_ref)
print(traj_load)

if traj_dcd_check == True:
  print("Trajectory concatenated successfully! :-)")
else:
  print("ERROR: Check your inputs! ")

# Congrats, you have finished Step 1 of the MD Colab! :-)

You will need two files for step 2. They will be in the google drive folder that you have put all your data in.

1. **.dcd** - You should find a .dcd file named "Jobname_nw.dcd". "Jobname" will be the name you assigned for your job earlier.
3. **.pdb** - You should find a similar file titled "Jobname_nw.pdb".

MAKE SURE THAT BOTH FILE NAMES INCLUDE "NW" IN THE NAME!! These files have the waters already removed (NW = no waters), and will make part 2 much easier.

Here is a link to step 2: https://colab.research.google.com/drive/1gsqVx570FsOm7-aDZNslvHc4Kkq2Gp1v?usp=

Here is a link to step 3 (if you need it): https://colab.research.google.com/drive/1HeOfR2AN601NK0r-0vcpkkkgKLZ5Pr6T?usp=drive_link